+ The client first request the public key of asymmetric encryption from the server
+ Client generates a key for symmetric encryption
+ Client uses public key obtained from server to encrypt the public key and send to server
+ Server uses its own private key to decrypted the received the encrypted key from client
+ After the above negotiation process, server and client will use use symmetric encryption to encrypt the following communication with the negotiated key

In [237]:
import socket
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import padding as pad
from cryptography.hazmat.primitives import hashes
import base64
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

In [238]:
# define a special request
class Request:
    # a simple request class containing method, url and headers
    def __init__(self, method: str, url: str, headers: {str: str}, request_data: str, body):
        self.method = method  # GET, POST, PUT, DELETE
        self.url = url  # e.g. /index.html
        self.headers = headers
        self.request_data = request_data
        self.body = body

    def convert(self):
        # output request
        # custom ENCRYPTION/1.1 request
        request = "{} {} ENCRYPTION/1.1\r\n".format(self.method, self.url)
        request += "\r\n".join("{}: {}".format(k, v) for k, v in self.headers.items())
        request += "\r\n\r\n" + self.body
        return request

In [239]:
# socket connect 127.0.0.1:8080/
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("127.0.0.1", 8080))

In [240]:
# first step: client request a public key
request = Request("GET", "/public_key", {}, "", "")
s.sendall(request.convert().encode("utf-8"))
while True:
    r = s.recv(1024)
    if r is not None:
        break
public_key = serialization.load_pem_public_key(
    r,
    backend=default_backend()
)
print("public key size: ", public_key.key_size)

public key size:  2048


In [241]:
# second step: Client generates a key for symmetric encryption
symmetric_key = os.urandom(32)
print('Key Generated', symmetric_key)

iv = os.urandom(16)
print('IV Generated', iv)

Key Generated b"#A\xbeN\xf1\x0c'iAa\x86\xa2\x9d`\x97o\xb4\xdc {\xb1.\x85#\x01?\x8dM\xe7;_!"
IV Generated b'\xd9\x8f-L\xc9\r\xd2-\xfbD\xf3\x0e\xee;6\x8e'


In [242]:
# third step: Client encrypts the symmetric key with the public key and sends it to the server

# encrypt the symmetric key with the public key
encrypted_key = public_key.encrypt(
    symmetric_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)
encrypted_symmetric_key = base64.b64encode(encrypted_key).decode("utf-8")
print('Key Encrypted', encrypted_symmetric_key)
print('Key Encrypted Length', len(encrypted_symmetric_key))

# send the encrypted symmetric key to the server
request = Request("POST", "/encrypted_symmetric_key", {"encrypted_symmetric_key": encrypted_symmetric_key}, "", "")
s.sendall(request.convert().encode("utf-8"))

encrypted_iv = public_key.encrypt(
    iv,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)
encrypted_iv = base64.b64encode(encrypted_iv).decode("utf-8")
print('IV Encrypted', encrypted_iv)
print('IV Encrypted Length', len(encrypted_iv))
request = Request("POST", "/encrypted_iv", {"encrypted_iv": encrypted_iv}, "", "")
s.sendall(request.convert().encode("utf-8"))

Key Encrypted XkXbhHBdosu7XZY9jEM2nzp60snsLt1ayzyKteojNHQ99Mk+45eCW7/6eF5zG/dfGesmVBS4eKCSoHUU5mYsrdUHMrlNqjvggu+8G/OeL6OqPrjcXVnbJijAoo1k8B0yxuRC8CxtkQ3LbvMDDQ4b5vJgmHCYnNNOwfHV5khR+8MKRMkbfYZOZj7lPs3tW+cdQroBZHh6+XCn+kMQpEqSqFQYA9q1eDHpX7IX+PrI4ri1BBYm55VyPg3SJtAO8uetB0ptcX5wJGHX2SSo8qgLdguvt2nBT7Nn9Ynkv0uGHA0zGGEc+caGTNXZDt9VzqBHlG08NYbrAzTJIravzbBmAQ==
Key Encrypted Length 344
IV Encrypted DhtTImwJLhjJWTd8eovuqMA2gcZ+h/Uid3CzON66Pw2DpGGRkeEMjcWfjYfRW5jSueiOIJ+9htoiMZh7LtMlK+jka8XS1qdeh7bGIj/1BUlvE/T3B3kAplRSDzhmpaZOAsP7WLdAo6/+3oXA38RzCGnB37V86Pb7yZDU9c2T93aopoc8U86wav0VKcFavwZpAHNhgQvIQAjSxUldnBSkIPg/qcdntZr1yY8NIjUdUj8nhiWrFtyfbVIysQt1NwvC3Sah6ALXLZO+ib04JzfXnnfA4sIBI7E8Ybx/oI0MDUrNdH3/yeRW+SOCZcQSYEmGAg+3Vh5ivMVBdjn3YED6qg==
IV Encrypted Length 344


In [243]:
body = """------WebKitFormBoundary7MA4YWxkTrZu0gW\r
Content-Disposition: form-data; name="firstfile"; filename="a.txt"\r
Content-Type: text/plain\r
\r
I love U!\r
------WebKitFormBoundary7MA4YWxkTrZu0gW--"""

# fourth step: Client encrypts the file with the symmetric key and sends it to the server
# encrypt the file with the symmetric key
cipher = Cipher(algorithms.AES(symmetric_key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()

# Pad the body to ensure it is a multiple of the block length
block_size = cipher.algorithm.block_size // 8
padder = pad.PKCS7(block_size * 8).padder()
padded_body = padder.update(body.encode('utf-8')) + padder.finalize()

encrypted_body = encryptor.update(padded_body) + encryptor.finalize()
encrypted_body = base64.b64encode(encrypted_body).decode("utf-8")
print('Body Encrypted', encrypted_body)

headers = {
    "Content-Type": "multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW",
    "Content-Length": len(encrypted_body),
    "Authorization": "Basic Y2xpZW50MToxMjM="
}

request = Request("POST", "/upload?path=client1/", headers, "", encrypted_body)
print(request.convert())
s.sendall(request.convert().encode("utf-8"))
while True:
    r = s.recv(1024)
    if r is not None:
        break

print(r.decode("utf-8"))

Body Encrypted LYDRQ0pKaS3cVVl3u0Rkzi9CtsZEGdTPIp1wV3Hd02OIH2XuzC9rN77482seIKgjWkFM7rGviBEVgmsIUcxzOLgdPxrmFD/v8PS3B0Zu8PNlOh2KTruM/W9R/tD2xxY4dxzty89Sfa0hkYNR+TKbERWAdfi1Q1KfrM2EbE9KzeHcX0+V34/7gJ6ftb4dbB1fJ4Nctg7jYwlJil2yLrXdiQ0JVaBSiflciSnK+EoYkEFXpV14UpPVQq85Q4eiKvUl
POST /upload?path=client1/ ENCRYPTION/1.1
Content-Type: multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW
Content-Length: 256
Authorization: Basic Y2xpZW50MToxMjM=

LYDRQ0pKaS3cVVl3u0Rkzi9CtsZEGdTPIp1wV3Hd02OIH2XuzC9rN77482seIKgjWkFM7rGviBEVgmsIUcxzOLgdPxrmFD/v8PS3B0Zu8PNlOh2KTruM/W9R/tD2xxY4dxzty89Sfa0hkYNR+TKbERWAdfi1Q1KfrM2EbE9KzeHcX0+V34/7gJ6ftb4dbB1fJ4Nctg7jYwlJil2yLrXdiQ0JVaBSiflciSnK+EoYkEFXpV14UpPVQq85Q4eiKvUl
HTTP/1.1 200 OK
Connection: keep-alive
Content-Type: text/html
Content-Length: 18

encrypted responseHTTP/1.1 200 OK
Connection: keep-alive
Content-Type: text/html
Content-Length: 18

encrypted response
